Unscented Kalman Filter algorithm:

Predict step: <br>
\begin{align}
X & = SigmaFunction(x, P) \\
W^m, W^c & = WeightFunction(n, parameters) \\
Y & = f(X) \\
x' & = \sum{w^mY} \\
P' & = \sum{w^c(Y-x')(Y-x')^T + Q} \\
\end{align}

Where:<br>


Update step: <br>
\begin{align}
Z & = h(Y) \\
u_z & = \sum{w^mZ} \\
y & = z - u_z \\
P_z & = \sum{w^c(Z-u_z)(Z-u_z)^T + R} \\
P_z & = [\sum{w^c(Y-x')(Y-x')^T}]P_z^{-1} \\
\end{align}

In [ ]:
import numpy as np


def sigma_function(x, P, n, alpha):
    k = n - 3

    l = alpha ** 2 * (n + k)

    X = list()
    X.append(x)

    for i range(1, n + 1):
        x_i = x + np.sqrt((n + l) @ P)[:, i]
        X.append(x_i)
    
    for i in range(n + 1, 2*n + 1):
        x_i = x - np.sqrt((n + l) @ P)[:, i - n]
        X.append(x_i)

    return X


def weight_function(n, alpha=1, betta=2):
    k = n - 3

    l = alpha ** 2 * (n + k)
    
    wcs = list()
    wms = list()

    wms.append(l/(n + l))
    wcs.append(l / (n + l) + 1 - alpha ** 2 + betta)

    for i in range(1, 2*n + 1):
        wcs.append(1 / (2 * (n + l)))
        wms.append(1 / (2 * (n + l)))

    return wms, wcs

In [ ]:
class UKF:
    def __init__(self, dim_x, dim_z, fs):
        pass

    def predict(self, u=0):
        X = sigma_function(self.x, self.P, self.dim_x, alpha=1.0)
        wm, wc = weight_function(self.dim_x, alpha=1.0, betta=2.0)
        Y = self.fs(X, u)
        x, P = self._unscented_transform(Y, wm, wc, self.Q, self.dim_x, self.x)
        self.x = x
        self.P = P

    def _unscented_transform(self, Y, wm, wc, Q, dim_x, x):
        x = 0
        P = np.zeros((dim_x, dim_x))

        for i in range(dim_x * 2 + 1):
            x += wm[i]*Y[i]
            P += wc[i] @ (Y[i] - x) @ (Y[i] - x).T + Q
        return x, P

    def update(self, z=0):
        pass